In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
macro=np.loadtxt('C:\\Users\\Takan\\Desktop\\Chicago_MAPSS\\Core Study\\Quarter2\\Structural Estimation\\PS3\\MacroSeries.txt',delimiter=',')

In [2]:
#macro=pd.DataFrame(macro,index=index,columns=['c','k','w','r'])
macro=pd.DataFrame(macro,columns=['c','k','w','r'])


In [3]:
w=macro.loc[:,'w']
k=macro.loc[:,'k']
w=macro.loc[:,'w']
r=macro.loc[:,'r']

In [4]:
#Define Normal Distribution
import numpy as np
import scipy as sp
def normal_pdf(xvals,alpha,mu,rho,sigma):
    pdf_vals=(np.exp(-(xvals-(rho*xvals.shift(1)+(1-rho)*mu))**2/(2*sigma**2)))/(2*np.pi*sigma**2)**0.5
    vals=pdf_vals[1:]
    return vals


In [5]:
#Define Log Likelihood for Normal Dist
def log_lik_normal(xvals,alpha,mu,rho,sigma):
    pdf_vals=normal_pdf(xvals,alpha,mu,rho,sigma)
    ln_pdf_vals=np.maximum(np.log(pdf_vals),np.log(1e-10))
    log_lik_val=ln_pdf_vals.sum()
    return log_lik_val

In [6]:
def crit(params):
    alpha,mu,rho,sigma=params
    z=np.log(w/((1-alpha)*k**alpha))
    log_lik_val=log_lik_normal(z,alpha,mu,rho,sigma)
    neg_lok_lik_val=-log_lik_val
    return neg_lok_lik_val


In [7]:
# Guesses for a,ρ,σ 
alpha_init_1=0.5
mu_init_1=2.0
rho_init_1=0.5
sigma_init_1=2.0

In [30]:
import scipy.optimize as opt
from scipy.optimize import minimize
bounds=((1e-10,0.99999),(1e-10,None),(-0.9999,0.9999),(1e-10,None))# I have to put bounds for the alpha and beta since they must remain positive
params_init_1= np.array([alpha_init_1,mu_init_1,rho_init_1,sigma_init_1]) #initial Parameters


results=opt.minimize(crit, params_init_1,bounds=bounds)
alpha_MLE,mu_MLE,rho_MLE,sigma_MLE = results.x
zbest=np.log(w/((1-alpha_MLE)*k**alpha_MLE)) #calculate the Zs using the optimized parameters
loglikelihoodnormal=log_lik_normal(zbest,alpha_MLE,mu_MLE,rho_MLE,sigma_MLE) #calculate the log likelihood of those Zs
print('alpha_MLE=', alpha_MLE, ' mu_MLE=', mu_MLE,'rho_MLE=',rho_MLE,'sigma_MLE=',sigma_MLE)

alpha_MLE= 0.462560409983  mu_MLE= 9.45352090826 rho_MLE= 0.716891495746 sigma_MLE= 0.0924532980388


C:\Users\Takan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [31]:
results

      fun: -95.24893982367408
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0081414 , -0.00057412, -0.00017764, -0.00548113])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 405
      nit: 39
   status: 0
  success: True
        x: array([ 0.46256041,  9.45352091,  0.7168915 ,  0.0924533 ])

In [32]:
invhess=results.hess_inv.todense()
invhess

array([[  4.49706907e+01,  -6.38781535e+02,  -4.47314620e+01,
         -4.09227889e+00],
       [ -6.38781535e+02,   9.07351249e+03,   6.35383077e+02,
          5.81296767e+01],
       [ -4.47314620e+01,   6.35383077e+02,   4.45024694e+01,
          4.07137215e+00],
       [ -4.09227889e+00,   5.81296767e+01,   4.07137215e+00,
          3.73086364e-01]])

In [33]:
results
OffDiagNeg = np.array([[1, -1,-1,-1],[-1, 1,-1,-1],[-1,-1,1,-1],[-1,-1,-1, 1]])
vcv_mle = invhess*OffDiagNeg
vcv_mle

array([[  4.49706907e+01,   6.38781535e+02,   4.47314620e+01,
          4.09227889e+00],
       [  6.38781535e+02,   9.07351249e+03,  -6.35383077e+02,
         -5.81296767e+01],
       [  4.47314620e+01,  -6.35383077e+02,   4.45024694e+01,
         -4.07137215e+00],
       [  4.09227889e+00,  -5.81296767e+01,  -4.07137215e+00,
          3.73086364e-01]])

In [34]:
stderr_alpha_mle = np.sqrt(vcv_mle[0,0])
stderr_mu_mle = np.sqrt(vcv_mle[1,1])
stderr_rho_mle = np.sqrt(vcv_mle[2,2])
stderr_sigma_mle = np.sqrt(vcv_mle[3,3])
print('VCV(MLE) = ', vcv_mle)
print('Standard error for alpha estimate = ', stderr_alpha_mle)
print('Standard error for mu estimate = ', stderr_mu_mle)
print('Standard error for rho estimate = ', stderr_rho_mle)
print('Standard error for sigma estimate = ', stderr_sigma_mle)

VCV(MLE) =  [[  4.49706907e+01   6.38781535e+02   4.47314620e+01   4.09227889e+00]
 [  6.38781535e+02   9.07351249e+03  -6.35383077e+02  -5.81296767e+01]
 [  4.47314620e+01  -6.35383077e+02   4.45024694e+01  -4.07137215e+00]
 [  4.09227889e+00  -5.81296767e+01  -4.07137215e+00   3.73086364e-01]]
Standard error for alpha estimate =  6.70601899012
Standard error for mu estimate =  95.2549866674
Standard error for rho estimate =  6.67101711928
Standard error for sigma estimate =  0.610807959779


In [35]:
#########   QUESTION B ######################

In [36]:
def crit_2(params):
    alpha,mu,rho,sigma=params
    z=np.log(r/(alpha*k**(alpha-1)))
    log_lik_val=log_lik_normal(z,alpha,mu,rho,sigma)
    neg_lok_lik_val=-log_lik_val
    return neg_lok_lik_val

In [37]:
# Guesses for a,ρ,σ 
alpha_init_2=0.5
mu_init_2=2.0
rho_init_2=0.5
sigma_init_2=2.0

In [38]:
import scipy.optimize as opt
from scipy.optimize import minimize
bounds=((1e-10,0.99999),(1e-10,None),(-0.9999,0.9999),(1e-10,None))# I have to put bounds for the alpha and beta since they must remain positive
params_init_2= np.array([alpha_init_2,mu_init_2,rho_init_2,sigma_init_2]) #initial Parameters

results_2=opt.minimize(crit_2, params_init_2,bounds=bounds)
alpha_MLE_2,mu_MLE_2,rho_MLE_2,sigma_MLE_2 = results.x
zbest_2=np.log(r/(alpha_MLE_2*k**(alpha_MLE_2-1)))
loglikelihoodnormal_2=log_lik_normal(zbest_2,alpha_MLE_2,mu_MLE_2,rho_MLE_2,sigma_MLE_2)
print('alpha_MLE_2=', alpha_MLE_2, ' mu_MLE_2=', mu_MLE_2,'rho_MLE_2=',rho_MLE_2,'sigma_MLE_2=',sigma_MLE_2)

C:\Users\Takan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


alpha_MLE_2= 0.462560409983  mu_MLE_2= 9.45352090826 rho_MLE_2= 0.716891495746 sigma_MLE_2= 0.0924532980388


In [39]:
results_2

      fun: -95.24893945236481
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  2.06341610e-03,   1.19371180e-04,  -2.41584530e-05,
         2.65742983e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 250
      nit: 31
   status: 0
  success: True
        x: array([ 0.71668497,  4.82387524,  0.4628098 ,  0.09245355])

In [40]:
#Comment : We have to note that our results are VERY sensitive to our initial guesses. We also note that the estimated parameters
#differ when we use different equations

In [41]:
invhess_2=results_2.hess_inv.todense()
invhess_2

array([[  2.09521297e+01,  -3.55901947e+02,  -5.33033993e+01,
         -1.03587842e+00],
       [ -3.55901947e+02,   6.04550528e+03,   9.05432787e+02,
          1.75957272e+01],
       [ -5.33033993e+01,   9.05432787e+02,   1.35647956e+02,
          2.63625185e+00],
       [ -1.03587842e+00,   1.75957272e+01,   2.63625185e+00,
          5.13362262e-02]])

In [42]:
results_2
OffDiagNeg = np.array([[1, -1,-1,-1],[-1, 1,-1,-1],[-1,-1,1,-1],[-1,-1,-1, 1]])
vcv_mle_2= invhess_2*OffDiagNeg
vcv_mle_2

array([[  2.09521297e+01,   3.55901947e+02,   5.33033993e+01,
          1.03587842e+00],
       [  3.55901947e+02,   6.04550528e+03,  -9.05432787e+02,
         -1.75957272e+01],
       [  5.33033993e+01,  -9.05432787e+02,   1.35647956e+02,
         -2.63625185e+00],
       [  1.03587842e+00,  -1.75957272e+01,  -2.63625185e+00,
          5.13362262e-02]])

In [43]:
stderr_alpha_mle_2 = np.sqrt(vcv_mle_2[0,0])
stderr_mu_mle_2 = np.sqrt(vcv_mle_2[1,1])
stderr_rho_mle_2 = np.sqrt(vcv_mle_2[2,2])
stderr_sigma_mle_2 = np.sqrt(vcv_mle_2[3,3])
print('VCV(MLE)_2 = ', vcv_mle_2)
print('Standard error for alpha estimate = ', stderr_alpha_mle_2)
print('Standard error for mu estimate = ', stderr_mu_mle_2)
print('Standard error for rho estimate = ', stderr_rho_mle_2)
print('Standard error for sigma estimate = ', stderr_sigma_mle_2)

VCV(MLE)_2 =  [[  2.09521297e+01   3.55901947e+02   5.33033993e+01   1.03587842e+00]
 [  3.55901947e+02   6.04550528e+03  -9.05432787e+02  -1.75957272e+01]
 [  5.33033993e+01  -9.05432787e+02   1.35647956e+02  -2.63625185e+00]
 [  1.03587842e+00  -1.75957272e+01  -2.63625185e+00   5.13362262e-02]]
Standard error for alpha estimate =  4.57734963243
Standard error for mu estimate =  77.7528474054
Standard error for rho estimate =  11.6468002583
Standard error for sigma estimate =  0.226574990339


In [44]:
#Question C.
zstar= z=np.log(1/(alpha_MLE*7500000**(alpha_MLE-1))) #Calculate Zstar that makes r=1
zstar
#we know that z is N(rho...etc) so we have zt-1 and we will use norm dist, the cumulative.

9.2788690978187631

In [45]:
#So Zt follows N(rho*Zt-1 +(1-rho)*mu,sigma**2)
#We know Zt-1 and we will also use the estimates that we obtained from Question 1
mean=rho_MLE*10+(1-rho_MLE)*mu_MLE
stdev=sigma_MLE
print(mean)
print(stdev)


9.84528712173
0.0924532980388


In [46]:
from scipy.stats import norm
prob=1-norm.cdf(zstar,mean,stdev)
prob

0.99999999955092223